# pydantic
Im vorherigen Kapitel haben wir `@dataclass`es angeschaut. Diese haben eine kurze und sehr praktische Möglichkeit angeboten, Klassen für die Speicherung von Daten zu erstellen.

Im Zusammenhang mit Daten müssen oft weitere Dinge gemacht werden, die sich immer wieder wiederholen:
* Die Struktur einer Klasse definieren (welche Variablen hat die Klasse).
* Daten (Werte) überprüfen, z.B. anhand des Datentypes.
* (De)Serialization: Daten umwandeln von oder nach JSON, XML oder YAML.

Pydantic widmet sich genau diesen Themen.

## Pydantic installieren

Pydantic ist nicht in der Standard-Installation von Python enthalten und muss daher zuerst heruntergeladen werden.

Um es zu installieren, füge das Package `pydantic` zu deinen Dependencies hinzu.

Verwendest du bereits Poetry (wird im Kapitel "project_structure" später erläutert), dann kannst du das mit folgendem Kommandozeilenbefehl tun:

In [ ]:
!poetry add pydantic

Verwendest du kein Poetry, dann ist eine Good-Practice, die verwendeten Dependencies in einer Datei zu notieren, damit andere Entwickler wissen, welche Dependencies/Packages sie auch benötigen.

Solche Dependencies werden oft in der Datei `requirements.txt` gespeichert, wobei jede Zeile der Name der Dependency ist (kann auch die Version enthalten).

In diesem Fall macht es Sinn, nicht die Dependency alleine zu installieren, sondern so, wie sie in der Dependency-Datei definiert ist. Das kann mit dem Befehl `pip install -r <Dependency-Dateiname>` (`-r` für "requirements") erreicht werden:

In [ ]:
! echo pydantic > ./requirements.txt

! pip install -r ./requirements.txt

Bist du hingegen ein kleiner Erfinder und unstrukturierter Bastler, dann reicht folgender Kommandozeilenbefehl aus:

In [ ]:
! pip install pydantic

## Datenklasse erstellen

Eine neue Datenklasse kann ziemlich ähnlich mit pydantic erstellt werden wie mit der `@dataclass`-Annotation. Zu Beachten ist, dass die neue Datenklasse (auch Model genannt) von der pydantic-Klasse `BaseModel` erben muss, sprich nach dem Namen der Klasse muss in Klammern `BaseModel` angegeben werden:

In [ ]:
from pydantic import BaseModel


class Tree(BaseModel):
    # Instanz-Variablen
    species: str
    height: float

    def describe(self):
        return f"This tree is a {self.species} and is {self.height} m tall."

Die Klasse kann dan ziemlich ähnlich wie eine `@dataclass`-Klasse verwendet werden.

Beachte, dass beim Konstruktor die Namen der Argumente angegeben werden muss:

In [ ]:
dracula_tree = Tree(species="Dracula Orchid", height=0.2)

dracula_tree.describe()


## Validierung
Pydantic wird oft auch dafür verwendet, um Python mit Features zu versehen, die typisierte Programmiersprachen oft schon mitbringen: Prüfen, ob der Datentyp bei einer Zuweisung stimmt.

Bei unserem Model (=Datenklasse) haben wir angegeben, dass die `height` vom Typ `float` ist. Pydantic berücksichtigt diese Information und wirft einen Fehler, wenn etwas anderes als eine Zahl versucht wird, zuzuweisen:

In [ ]:
dracula_tree = Tree(species="Baobab Tree", height="twenty")

Zusätzlich ist es möglich, weitere Bedingungen an die Variablen zu knüpfen, bspw. dass die Höhe grösser als `0` sein muss. Hierfür weisen wir dem Feld den Rückgabewert der Funktion `Field(...)` zu:

In [ ]:
from pydantic import BaseModel, Field

class Tree(BaseModel):
    species: str = Field(..., description="The species of the tree.")
    height: float = Field(..., gt=0, description="The height of the tree in meters.")


Das `description`-Argument hat keine Funkion. Sie wird oft dafür verwendet, um den Sinn und die Verwendung der Variable zu dokumentieren, damit ein andere(r) Entwickler:in versteht, wofür die Variable verwendet wird.

Für die Variable `height` wurde das Argument `gt` spezifiziert. Dies steht für "greater than" (also grösser als). Mit diesem optionalen Argument können wir festlegen, dass keine Werte unter 0 akzeptiert werden.

Folglich sollte folgendes funktionieren:

In [ ]:
pancake_tree = Tree(species="Pancake Tree", height=13)

Und folgendes natürlich nicht:

In [ ]:
dwarf_willow_tree = Tree(species="Dwarf Willow", height=-0.02)

Neben `gt` gibt es viele weitere Parameter, die hilfreich im Bezug auf Validierung sind. Hier sind einige aufgelistet:
* `gt`: grösser als.
* `ge`: Grösser oder gleich.
* `lt`: Kleiner als.
* `le`: Kleiner oder gleich.
* `ne`: ungleich.
* `anystr_length`: Länge eines Strings.
* `regex`: Regex, der die Variable "matchen" muss.
* `email`: Muss eine Email sein.
* `url`: Muss eine URL sein.
* `positive`: Muss eine positive Zahl sein.
* `negative`: Muss eine negative Zahl sein.
* `none`: Nur `None` ist als Wert zulässig.

Hier findest du mehr Parameter und Beispiele: https://docs.pydantic.dev/latest/usage/schema/#field-customization-parameters

## Serialization

Unter diesem Begriff verstehen wir das Umwandeln vom Objekt in ein "serialisiertes" Format wie JSON, XML oder YAML. Pydantic unterstützt das Konvertieren in JSON.

Nehmen wir als Beispiel folgendes als Objekt:

In [ ]:
corkscrew_tree = Tree(species="Corkscrew Willow", height=7)

Möchten wir das Objekt als Dictionary präsentiert haben, dann bekommen wir dies mit der `json()`-Methode:

In [ ]:
tree_as_dict = corkscrew_tree.dict()
tree_as_dict

Wenn wir es direkt als JSON-String haben möchten, dann reicht dies aus:

In [ ]:
tree_as_json = corkscrew_tree.json()
tree_as_json

## Deserialization
Deserialization ist Serialization in die andere Richtung:

Wir haben z.B. einen JSON-String und möchten daraus ein Objekt instantiieren:

In [ ]:
json_string = '{"species": "Elephant Foot Yam Tree", "height": 2.5}'

elephant_tree = Tree.parse_raw(json_string)

elephant_tree

Und wenn wir von einem Dictionary aus deserialisieren möchten:

In [ ]:
dictionary = {"species": "Tickle-Me-Not Tree", "height": 1.5}

tickle_me_not_tree = Tree.parse_obj(dictionary)

tickle_me_not_tree